Theo Hinton-Hallows Bias In AI Coursework Code

Import data from excel file

In [459]:
import pandas as pd

df = pd.read_excel(r"recruitment.xls",)

df.loc[df['OfferNY'].isnull(), 'OfferNY'] = 0



Task 1

Question 1

In [460]:
# Gender

g1 = df[df['Gender'] == 1]
g2 = df[df['Gender'] == 2]

g1_offered = g1[g1['OfferNY'] == 1].shape[0]
g2_offered = g2[g2['OfferNY'] == 1].shape[0]
print('Proportion of gender 1 given an offer', g1_offered/g1.shape[0])
print('Proportion of gender 2 given an offer', g2_offered/g2.shape[0])

# Race

b1 = df[df['BAMEyn'] == 1]
b2 = df[df['BAMEyn'] == 2]

b1_offered = b1[b1['OfferNY'] == 1].shape[0]
b2_offered = b2[b2['OfferNY'] == 1].shape[0]
print('Proportion of BAMEyn 1 given an offer', b1_offered/b1.shape[0])
print('Proportion of BAMEyn 2 given an offer', b2_offered/b2.shape[0])

Proportion of gender 1 given an offer 0.23076923076923078
Proportion of gender 2 given an offer 0.04950495049504951
Proportion of BAMEyn 1 given an offer 0.06611570247933884
Proportion of BAMEyn 2 given an offer 0.12578616352201258


Question 2

In [461]:

# Gets 9 statistics for the specified group of the specified sensitive attribute from the data
def get_stats(attr, grp):
    filtered = df[df[attr] == grp]
    num_grp = filtered.shape[0]
    num_offered = filtered[filtered['OfferNY'] == 1].shape[0]
    num_not_offered = filtered[filtered['OfferNY'] == 0].shape[0]
    num_short = filtered[filtered['ShortlistedNY'] == 1].shape[0]
    num_not_short = filtered[filtered['ShortlistedNY'] == 0].shape[0]
    stats = [num_grp, num_offered, num_not_offered, num_offered/num_grp, num_not_offered/num_grp, num_short, num_not_short, num_short/num_grp, num_not_short/num_grp]
    #stats corresponds to rows in table in pdf report in q2:
    #[Total, Total given offers, Total not given offers, Proportion given offers, Proportion not given offers, Total shortlisted for interview, Total not shortlisted for interview, Proportion shortlisted for interview, Proportion not shortlisted for interview]
    return stats

#Male
print(get_stats('Gender', 1))
#Female
print(get_stats('Gender', 2))
#BAME
print(get_stats('BAMEyn', 1))
#Not BAME
print(get_stats('BAMEyn', 2))


[78, 18, 60, 0.23076923076923078, 0.7692307692307693, 38, 40, 0.48717948717948717, 0.5128205128205128]
[202, 10, 192, 0.04950495049504951, 0.9504950495049505, 50, 152, 0.24752475247524752, 0.7524752475247525]
[121, 8, 113, 0.06611570247933884, 0.9338842975206612, 19, 102, 0.15702479338842976, 0.8429752066115702]
[159, 20, 139, 0.12578616352201258, 0.8742138364779874, 69, 90, 0.4339622641509434, 0.5660377358490566]


Question 5

In [462]:
from scipy.stats import chi2 # Needed for chi-squared cdf

# Performs a Chi-Squared test for goodness of fit to test the independence of one of the sensitive attributes and the shortlisting process
def chi_squared(attr, sig_level):
    s1 = get_stats(attr, 1) #see q2
    s2 = get_stats(attr, 2)
    observed_table = [[s1[5], s2[5], s1[5]+s2[5]],
                      [s1[6], s2[6], s1[6]+s2[6]],
                      [s1[0], s2[0], s1[0]+s2[0]]]
    """
    observed table layed out like this:
                    |Grp1   |Grp2   |Total
    shortlisted     |   x   |   x   |   x
    not shortlisted |   x   |   x   |   x
    Total           |   x   |   x   |   x
    """
    expected_table = [[observed_table[2][j]*observed_table[i][2]/observed_table[2][2] for j in range(2)] for i in range(2)] # Gets table of corresponding expected values of each of the non-total squares in the observed table assuming null-hypothesis
    test_statistic = 0
    for i in range(2):
        for j in range(2):
            test_statistic += ((observed_table[i][j] - expected_table[i][j])**2)/expected_table[i][j]
    
    p_value = 1-chi2.cdf(test_statistic, 1) # Chi-squared cdf with 1 degree of freedom gives lower tail and we want upper tail so we subtract it from 1
    if p_value < sig_level:
        print(f'Reject H0 - {attr} and getting shortlisted are not independent')
    else:
        print(f'Do not reject H0 - insufficient evidence to suggest {attr} and getting shortlisted are not independent')
    print(f'Test Statistic: {test_statistic} P-Value: {p_value}')

chi_squared('Gender', 0.05)
chi_squared('BAMEyn', 0.05)

Reject H0 - Gender and getting shortlisted are not independent
Test Statistic: 14.996576580734997 P-Value: 0.00010770639139778826
Reject H0 - BAMEyn and getting shortlisted are not independent
Test Statistic: 24.452319861644675 P-Value: 7.617173791718557e-07


Task 2

Question 1 and 2



In [463]:
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import roc_auc_score
import numpy as np
import math

#Preprocessing

# df.loc[df['OfferNY'].isnull(), 'OfferNY'] = 0 
# thats already been done but im putting it here again commented to point out its part of preprocessing


df.loc[df['FemaleONpanel'].isnull(), 'FemaleONpanel'] = 0 # add an option for when no interview takes place (just so the missing values dont break anything)
data = df.to_numpy() # only cos im more familiar with numpy arrays

y = data[:,6]
X = np.delete(data, [0, 6, 7, 8], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=9)


# Creating model

# Using custom implementation of logistic regression so i can modify it to include the adversarial bias mitigation
class CustomLogisticRegressor(BaseEstimator, ClassifierMixin):
    def __init__(self, rate=0.1, max_iters = 1000):
        self.rate = rate
        self.max_iters = max_iters

    # Training function
    def fit(self, X, y):
        X, y = check_X_y(X, y)

        self.classes_ = unique_labels(y)

        self.X_ = X
        self.y_ = y

        self.nFeatures = X.shape[0]
        self.nWeights = X.shape[1]

        self.weights_ = np.array([1 for _ in range(self.nWeights)], dtype = 'float64')


        # Gradient Descent
        self.cont = True
        self.iter = 0
        while self.iter<self.max_iters and self.cont:
            self.iter += 1
            self.cont = False
            temp = [0 for _ in range(self.nWeights)]
            for j in range(self.nWeights):
                delta = 0
                for i in range(self.nFeatures):
                    delta += (1/(1+math.e**(-np.dot(self.weights_, X[i])))-y[i])*X[i][j]
                delta = delta/self.nFeatures
                if delta != 0:
                    self.cont = True
                temp[j] = -self.rate * delta
            self.weights_ += temp

    def predict(self, X):
        check_is_fitted(self)

        X = check_array(X)

        boundary = np.matmul(X, self.weights_)
        for i in range(boundary.shape[0]):
            boundary[i] = np.round(1/(1+math.e**(-boundary[i])))
        return boundary
    
    def decision_function(self, X):
        check_is_fitted(self)

        X = check_array(X)
        boundary = np.matmul(X, self.weights_)
        for i in range(boundary.shape[0]):
            boundary[i] = 1/(1+math.e**(-boundary[i]))
        return boundary
    # Functions for compatibility with sklearn stuff which i may not end up needing
    def get_params(self, deep=True):
        return {"rate": self.rate, "max_iters": self.max_iters}
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

# Training

alpha = 0.5
max_iters = 100

regr = CustomLogisticRegressor(alpha, max_iters)

regr.fit(X_train, y_train)


# QUESTION 2


# Testing
corrects = 0
conf_mat = [[0,0],[0,0]]
y_pred = regr.predict(X_test)
y_score = regr.decision_function(X_test)
for i in range(y_test.shape[0]):
    if y_pred[i] == 1:
        if y_test[i] == 1:
            conf_mat[0][0] += 1
            corrects += 1
        else:
            conf_mat[0][1] += 1
    else:
        if y_test[i] == 1:
            conf_mat[1][0] += 1
        else:
            conf_mat[1][1] += 1
            corrects += 1

accuracy = corrects/y_test.shape[0]
precision = conf_mat[0][0]/(conf_mat[0][0]+conf_mat[0][1])
npv = conf_mat[1][1]/(conf_mat[1][1]+conf_mat[1][0])
recall = conf_mat[0][0]/(conf_mat[0][0]+conf_mat[1][0])
selectivity = conf_mat[1][1]/(conf_mat[1][1]+conf_mat[0][1])
f_1_score = precision*recall*2/(precision+recall)
roc_auc = roc_auc_score(y_test, y_score)

print(corrects, y_test.shape[0], accuracy)
print(conf_mat)
print(precision, npv, recall, selectivity, f_1_score, roc_auc)

# Fairness evaluation

# Gender

g1 = 0
g1p1 = 0
g2 = 0
g2p1 = 0

# Race
r1 = 0
r1p1 = 0
r2 = 0
r2p1 = 0
for i in range(y_test.shape[0]):
    if y_test[i] == 1:
        if X_test[i][0] == 1:
            g1 += 1
            if y_pred[i] == 1:
                g1p1 += 1
        else:
            g2 += 1
            if y_pred[i] == 1:
                g2p1 += 1
        if X_test[i][1] == 1:
            r1 += 1
            if y_pred[i] == 1:
                r1p1 += 1
        else:
            r2 += 1
            if y_pred[i] == 1:
                r2p1 += 1
gp1 = g1p1/g1
gp2 = g2p1/g2

rp1 = r1p1/r1
rp2 = r2p1/r2

print(gp1, gp2)
print(rp1, rp2)



77 84 0.9166666666666666
[[4, 2], [5, 73]]
0.6666666666666666 0.9358974358974359 0.4444444444444444 0.9733333333333334 0.5333333333333333 0.9777777777777777
0.5714285714285714 0.0
1.0 0.375


Question 3

In [464]:
# I will modify a copy of the CustomLogisticRegressor class I made above to include the adversarial model
# This code will implement one adversary for each sensitive attribute

class Adversary:
    def __init__(self, S, inp, rate):
        self.S_ = S
        self.inp_ = inp
        self.nFeatures = inp.shape[0]
        self.nWeights = inp.shape[1]
        self.rate = rate
        self.weights_ = np.array([1 for _ in range(self.nWeights)], dtype='float64')
    
    def update(self, new_inp):
        self.inp_ = new_inp
        temp = [0 for _ in range(self.nWeights)]
        for j in range(self.nWeights):
            delta = 0
            for i in range(self.nFeatures):
                delta += (1 + 1/(1+math.e**(-np.dot(self.weights_, self.inp_[i])))-self.S_[i])*self.inp_[i][j] # adding 1 at the start as both the sensitive attributes are either 1 or 2 rather than 0 or 1
            delta = delta/self.nFeatures
            temp[j] = -self.rate*delta
        self.weights_ += temp
    
    def pred(self, inp):
        return 1/(1+math.e**(-np.dot(self.weights_, inp)))


class AdversarialLogisticRegressor(BaseEstimator, ClassifierMixin):
    def __init__(self, rate=0.1, max_iters = 1000, alpha = 0.1):
        self.rate = rate
        self.max_iters = max_iters
        self.alpha = alpha

    # Training function
    def fit(self, X, y, sens):
        X, y = check_X_y(X, y)

        self.classes_ = unique_labels(y)

        self.X_ = X
        self.y_ = y

        self.nFeatures = X.shape[0]
        self.nWeights = X.shape[1]

        self.weights_ = np.array([1 for _ in range(self.nWeights)], dtype = 'float64')

        self.adversaries = []
        initial_ad_inp = np.array([y, self.decision_function(X)]).transpose() # Equality of odds, so adversary given true values of y and predicted values of y
        for i in sens:
            self.adversaries.append(Adversary(X[:,i], initial_ad_inp, self.rate))

        # Gradient Descent
        self.cont = True
        self.iter = 0
        while self.iter<self.max_iters and self.cont:
            self.iter += 1
            self.cont = False
            g = np.array([0 for _ in range(self.nWeights)], dtype='float64')
            for j in range(self.nWeights):
                deltag = 0
                for i in range(self.nFeatures):
                    deltag += (1/(1+math.e**(-np.dot(self.weights_, X[i])))-y[i])*X[i][j]
                deltag = deltag/self.nFeatures
                g[j] = deltag
            dweights = np.array([0 for _ in range(self.nWeights)], dtype = 'float64')
            dweights += g
            for x in range(len(self.adversaries)):
                h = np.array([0 for _ in range(self.nWeights)], dtype='float64')
                for j in range(self.nWeights):
                    deltah = 0
                    for i in range(self.nFeatures):
                        deltah += self.adversaries[x].weights_[1] * X[i][j] * math.e**(-np.dot(self.weights_, X[i])) * (self.adversaries[x].inp_[i][1]**2) * (1 + self.adversaries[x].pred(np.array([y[i],self.adversaries[x].inp_[i][1]])) - self.adversaries[x].S_[i])
                    deltah = deltah/self.nFeatures
                    h[j] = -deltah
                    # The above computes the derivative of the adversary loss function with respect to self.weights_[j]
                if np.dot(h, h) == 0:
                    proj = np.array([0 for _ in range(self.nWeights)], dtype='float64')
                else:
                    proj = -(np.dot(g, h)/np.dot(h, h))*h
                dweights += proj + self.alpha*h
            for i in range(self.nWeights):
                if dweights[i] != 0:
                    self.cont = True

            self.weights_ += self.rate * dweights
            new_ad_inp = np.array([y, self.decision_function(X)]).transpose()
            for x in range(len(self.adversaries)):
                self.adversaries[x].update(new_ad_inp)

    def predict(self, X):
        check_is_fitted(self)

        X = check_array(X)

        boundary = np.matmul(X, self.weights_)
        for i in range(boundary.shape[0]):
            boundary[i] = np.round(1/(1+math.e**(-boundary[i])))
        return boundary
    
    def decision_function(self, X):
        boundary = np.matmul(X, self.weights_)
        for i in range(boundary.shape[0]):
            boundary[i] = 1/(1+math.e**(-boundary[i]))
        return boundary
    # Functions for compatibility with sklearn stuff which i may not end up needing
    def get_params(self, deep=True):
        return {"rate": self.rate, "max_iters": self.max_iters}
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

rate = 0.2
max_iters = 1000
alpha = 0.5

adreg = AdversarialLogisticRegressor(rate, max_iters, alpha)
adreg.fit(X_train, y_train, [0, 1])



Question 4

In [465]:
corrects = 0
conf_mat = [[0,0],[0,0]]
y_pred = adreg.predict(X_test)
y_score = adreg.decision_function(X_test)
interviewed_but_predicted_0 = 0
for i in range(y_test.shape[0]):
    if y_pred[i] == 1:
        if y_test[i] == 1:
            conf_mat[0][0] += 1
            corrects += 1
        else:
            conf_mat[0][1] += 1
    else:
        if X_test[i][3] == 1:
            interviewed_but_predicted_0 += 1
        if y_test[i] == 1:
            conf_mat[1][0] += 1
        else:
            conf_mat[1][1] += 1
            corrects += 1

print(adreg.weights_)
accuracy = corrects/y_test.shape[0]
precision = conf_mat[0][0]/(conf_mat[0][0]+conf_mat[0][1])
npv = conf_mat[1][1]/(conf_mat[1][1]+conf_mat[1][0])
recall = conf_mat[0][0]/(conf_mat[0][0]+conf_mat[1][0])
selectivity = conf_mat[1][1]/(conf_mat[1][1]+conf_mat[0][1])
f_1_score = precision*recall*2/(precision+recall)
roc_auc = roc_auc_score(y_test, y_score)

print(corrects, y_test.shape[0], accuracy)
print(conf_mat)
print(interviewed_but_predicted_0)
print(precision, npv, recall, selectivity, f_1_score, roc_auc)


# Fairness evaluation

# Offer actually received

# Gender

g1 = 0
g1p1 = 0
g2 = 0
g2p1 = 0

# Race
r1 = 0
r1p1 = 0
r2 = 0
r2p1 = 0
for i in range(y_test.shape[0]):
    if y_test[i] == 1:
        if X_test[i][0] == 1:
            g1 += 1
            if y_pred[i] == 1:
                g1p1 += 1
        else:
            g2 += 1
            if y_pred[i] == 1:
                g2p1 += 1
        if X_test[i][1] == 1:
            r1 += 1
            if y_pred[i] == 1:
                r1p1 += 1
        else:
            r2 += 1
            if y_pred[i] == 1:
                r2p1 += 1
gp1 = g1p1/g1
gp2 = g2p1/g2

rp1 = r1p1/r1
rp2 = r2p1/r2

print(gp1, gp2)
print(rp1, rp2)


[-7.36999053e+00  1.07768529e-02  4.25489633e+01  1.82178842e+01
  2.32969059e+01]
65 84 0.7738095238095238
[[9, 19], [0, 56]]
0
0.32142857142857145 1.0 1.0 0.7466666666666667 0.4864864864864865 0.9333333333333333
1.0 1.0
1.0 1.0
